## Hubble Ultra Deep Field Image Detection

In [ ]:
%matplotlib inline
import numpy as np
import sep
import astropy
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['figure.figsize'] = [10., 8.]

### Open the FITS file

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
data = fits.getdata(fname)
data = data.byteswap().newbyteorder()

### Show the dimensions of the image

In [ ]:
print(type(data))
print(data.shape)

### Show the data

In [ ]:
m = np.mean(data)
s = np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

### Subtract the background

In [ ]:
bkg = sep.Background(data)

#get "global" mean and noise of the image
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate background as 2-d array, same size as original image
bkg_image = np.array(bkg)

#show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

In [ ]:
#evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

#show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

In [ ]:
#Subtract the background
data_sub = data-bkg

### Object Detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

#show how many objects were detected
print("There are", len(objects), "objects detected in this image.")

In [ ]:
from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
m = np.mean(data_sub)
s = np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin = m-s, vmax = m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width = 6*objects['a'][i], height = 6*objects['b'][i], angle=objects['theta'][i]*180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
#available fields
objects.dtype.names

### Aperture Photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

#show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
#what is the flux?
print("Flux:", flux)

### Convert flux to AB magnitude units

In [ ]:
flux = -2.5*np.log10(flux/3631)
print("Flux in AB Magnitude:", flux)

### Plotting the histogram

In [ ]:
histogram = plt.hist(flux, bins='auto')
plt.xlabel('x')
plt.ylabel('y')

## EXTRA CREDIT: Creating the RGB Image

### Create the RGB false image using "make_lupton_rgb"

In [ ]:
from astropy.visualization import make_lupton_rgb

#get the three FITS files and the data from them
image105 = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
f105w_data = fits.getdata(image105)
image125 = "hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits"
f125w_data = fits.getdata(image125)
image160 = "hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits"
f160w_data = fits.getdata(image160)

#create the false image
rgb = make_lupton_rgb(f160w_data, f125w_data, f105w_data, Q=0.2, stretch=0.005, filename='hudf_rgb_image.png')

plt.imshow(rgb, origin='lower')
plt.colorbar()

### Zoom into the image

In [ ]:
#just to see the pretty image :D
rgb = make_lupton_rgb(f160w_data[1000:2500,1000:2500], f125w_data[1000:2500,1000:2500], f105w_data[1000:2500,1000:2500], Q=0.2, stretch=0.005)

plt.imshow(rgb, origin='lower')
plt.colorbar()